In [0]:
!pip install corner

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
cd drive/My\ Drive/Projects/Autoencoder_1

/content/drive/My Drive/Projects/Autoencoder_1


In [0]:
ls

find_best_model.py      nn_utils.py                   README.md
models/                 plot_activation_functions.py  results.csv
my_matplotlib_rcparams  plots/                        run_model.py
my_matplotlib_style.py  processed_data/               utils.py
my_utils.py             __pycache__/


In [0]:
import os
import utils
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from scipy import stats
from tqdm import tqdm_notebook
import corner.corner as corner
import matplotlib.pyplot as plt
from hyperopt import fmin, tpe, hp, STATUS_OK

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [0]:
import fastai
from fastai import train as tr 
from fastai.callbacks import ActivationStats
from fastai import data_block, basic_train, basic_data

In [0]:
from my_utils import make_plots, load_data, train_evaluate_model
from utils import plot_activations

In [0]:
if torch.cuda.is_available():
    fastai.torch_core.defaults.device = 'cuda'

In [0]:
class AE_3D_200_custom(nn.Module):
    def __init__(self, hidden_dim_1, hidden_dim_2, hidden_dim_3, n_features=4):
        super(AE_3D_200_custom, self).__init__()
        self.en1 = nn.Linear(n_features, hidden_dim_1)
        self.en2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.en3 = nn.Linear(hidden_dim_2, hidden_dim_3)
        self.en4 = nn.Linear(hidden_dim_3, 3)
        self.de1 = nn.Linear(3, hidden_dim_3)
        self.de2 = nn.Linear(hidden_dim_3, hidden_dim_2)
        self.de3 = nn.Linear(hidden_dim_2, hidden_dim_1)
        self.de4 = nn.Linear(hidden_dim_1, n_features)
        self.relu = nn.ReLU()

    def encode(self, x):
        return self.en4(self.relu(self.en3(self.relu(self.en2(self.relu(self.en1(x)))))))

    def decode(self, x):
        return self.de4(self.relu(self.de3(self.relu(self.de2(self.relu(self.de1(self.relu(x))))))))

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

    def describe(self):
        return 'in-' + str(hidden_dim_1) + '-' + str(hidden_dim_2) + '-' + str(hidden_dim_3) + '-' +'3-' + str(hidden_dim_3) + '-' + str(hidden_dim_2) + '-' + str(hidden_dim_1) + '-out'

In [0]:
global best_loss
best_loss = 10000

In [0]:
def gridSearch(space):
        global best_hidden_dim_1, best_hidden_dim_2, best_hidden_dim_3, best_loss
        hidden_dim_1 = int(space['hidden_dim_1'])
        hidden_dim_2 = int(space['hidden_dim_2'])
        hidden_dim_3 = int(space['hidden_dim_3'])

        model = AE_3D_200_custom(hidden_dim_1, hidden_dim_2, hidden_dim_3)
        model_name = 'AE_3D_200_custom'
        num_epochs = 50
        learning_rate = 3e-4
        loss = train_evaluate_model(model, model_name, num_epochs, learning_rate, hidden_dim_1, hidden_dim_2, hidden_dim_3)

        if (loss < best_loss):
            best_loss = loss
            best_hidden_dim_1 = hidden_dim_1
            best_hidden_dim_2 = hidden_dim_2
            best_hidden_dim_3 = hidden_dim_3
            print('best_loss = {}'.format(best_loss))
            print('best_hidden_dim_1 = {}'.format(hidden_dim_1))
            print('best_hidden_dim_2 = {}'.format(hidden_dim_2))
            print('best_hidden_dim_3 = {}'.format(hidden_dim_3))

        return {'loss': loss, 'status': STATUS_OK }

In [0]:
space = {
'hidden_dim_1': hp.quniform('hidden_dim_1', 50, 200, 50),
'hidden_dim_2': hp.quniform('hidden_dim_2', 100, 400, 100),
'hidden_dim_3': hp.quniform('hidden_dim_3', 50, 200, 50),    
}

best_scores = fmin(fn=gridSearch, space=space, algo=tpe.suggest, max_evals=100)

  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]

epoch,train_loss,valid_loss,time
0,0.028870,0.028819,00:03
1,0.026430,0.027304,00:03
2,0.024298,0.025768,00:03
3,0.023960,0.024803,00:03
4,0.022365,0.021657,00:04
5,0.020623,0.020401,00:04
6,0.019990,0.019797,00:03
7,0.018621,0.020108,00:03
8,0.017999,0.018309,00:03
9,0.017799,0.017952,00:04


best_loss = 0.010315189138054848
best_hidden_dim_1 = 150
best_hidden_dim_2 = 300
best_hidden_dim_3 = 50
  1%|          | 1/100 [03:31<5:49:23, 211.75s/it, best loss: 0.010315189138054848]

epoch,train_loss,valid_loss,time
0,0.123886,0.120292,00:04
1,0.099298,0.097623,00:04
2,0.088118,0.085800,00:04
3,0.078471,0.078627,00:04
4,0.073561,0.075776,00:04
5,0.072635,0.070906,00:04
6,0.067961,0.068355,00:04
7,0.067781,0.067425,00:03
8,0.066094,0.064923,00:04
9,0.064377,0.068415,00:04


In [0]:
print('best_scores = {}'.format(best_scores))
print('best_loss = {}'.format(best_loss))